In [1]:
import requests
from collections import defaultdict
import pandas as pd
import numpy as np

##  APIs

1. Forest: AG.LND.FRST.ZS
2. Permanent cropland: AG.LND.CROP.ZS
3. Agricultural Land: AG.LND.AGRI.ZS
4. Arable Land: AG.LND.ARBL.ZS
5. Land Cereal production : AG.LND.CREL.HA
6. Rural population: SP.RUR.TOTL.ZS
7. Urban Population:SP.URB.TOTL.IN.ZS
8. Male Employment:SL.AGR.EMPL.MA.ZS
9. Female Employement: SL.AGR.EMPL.FE.ZS
10. Fertilizer Use: AG.CON.FERT.ZS
11. Cereal Production: AG.YLD.CREL.KG
12. % of GDP value: NV.AGR.TOTL.ZS
13. Total Population: SP.POP.TOTL
14. Under 5 mortality: SH.DYN.MORT
15. Greenhouse Gases: EN.ATM.GHGT.KT.CE
16. CO2 emissions: EN.ATM.CO2E.KT
17. Poverty : SI.POV.DDAY

In [ ]:
country_list = ['World', 'Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Channel Islands', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Polynesia', 'Gabon', 'Gambia, The', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong SAR, China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Rep.', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.', 'Kosovo', 'Kuwait', 'Kyrgyz Republic', 'Lao PDR', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao SAR, China', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania', 'Mauritius', 'Mexico', 'Micronesia, Fed. Sts.', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Romania', 'Russian Federation', 'Rwanda', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovak Republic', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'St. Kitts and Nevis', 'St. Lucia', 'St. Martin (French part)', 'St. Vincent and the Grenadines', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkiye', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, RB', 'Vietnam', 'Virgin Islands (U.S.)', 'West Bank and Gaza', 'Yemen, Rep.', 'Zambia', 'Zimbabwe']

In [ ]:
%store largest_10_pop_df
%store countries_population_df

Stored 'largest_10_pop_df' (DataFrame)
Stored 'countries_population_df' (DataFrame)


In [5]:
def removeFormatColumns(df):
    """
    _summary_

    Args:
        df (DataFrame): _description_

    Returns:
        DataFrame: _description_
    """
    df.drop(columns=['indicator','obs_status','decimal','countryiso3code', 'unit'], inplace=True, axis=1)

    return df

def formatColumns(df):
    """
    _summary_

    Args:
        df (DataFrame): _description_
    Returns:
        df (DateFrame):
    """
    
    df["date"] = pd.to_datetime(df["date"])

    #turn country feature into just country name
    for i, country in enumerate(df['country']):
        df.loc[i,'country'] = country['value']

    return df

def justCountries(df):
    """
    _summary_
    Args:
        df (DataFrame): _description_
    Returns:
        DataFrame: _description_
    """
    df=df[df['country'].isin(country_list)]
    return df

In [14]:
Land_API = ['SP.POP.TOTL', 'AG.LND.FRST.ZS','AG.LND.CROP.ZS','AG.LND.AGRI.ZS','AG.LND.ARBL.ZS','AG.LND.CREL.HA', 'SP.RUR.TOTL.ZS','SP.URB.TOTL.IN.ZS', 
'SL.AGR.EMPL.MA.ZS', 'SL.AGR.EMPL.FE.ZS', 'AG.CON.FERT.ZS','AG.YLD.CREL.KG', 
'NV.AGR.TOTL.ZS', 'SH.DYN.MORT', 'EN.ATM.GHGT.KT.CE', 'EN.ATM.CO2E.KT',         'SI.POV.DDAY']
urls = []
for indicator in Land_API:
    url = 'http://api.worldbank.org/v2/countries/indicators/' + indicator 
    urls.append(url)
dataframe_list = []

for url in urls:
    data = []
    try:  
        for page in range(1,18):
            payload = {'format': 'json', 'per_page': '1000', 'date':'1960:2022', 'page':page}     
            r = requests.get(url, params=payload)
            data+=r.json()[1]
        
        dataframe_list.append(pd.DataFrame(data))

    except:
        print('could not load data', url)

In [15]:
land_api_df = None

#format and combine datframes into one
for i, df in enumerate(dataframe_list):
  dataframe_list[i] = removeFormatColumns(df)
  dataframe_list[i] = formatColumns(df)
  dataframe_list[i] = justCountries(df)
  
  if land_api_df is not None:
    land_api_df.insert(loc=len(land_api_df.columns),column=Land_API[i], 
    value=df['value'])
  else:
    land_api_df = pd.DataFrame(df)
    land_api_df.rename(columns={'value' : Land_API[i]}, inplace=True)

In [34]:


urls = []
for indicator in living_condition_api:
    url = 'http://api.worldbank.org/v2/countries/indicators/' + indicator 
    urls.append(url)
dataframe_list = []

for url in urls:
    data = []
    try:  
        for page in range(1,18):
            payload = {'format': 'json', 'per_page': '1000', 'date':'1960:2022', 'page':page}     
            r = requests.get(url, params=payload)
            data+=r.json()[1]
        
        dataframe_list.append(pd.DataFrame(data))

    except:
        print('could not load data', url)

In [35]:
living_condition_df = None

#format and combine datframes into one
for i, df in enumerate(dataframe_list):
  dataframe_list[i] = removeFormatColumns(df)
  dataframe_list[i] = formatColumns(df)
  dataframe_list[i] = justCountries(df)
  
  if living_condition_df is not None:
    living_condition_df.insert(loc=len(living_condition_df.columns),column=living_condition_api[i], 
    value=df['value'])
  else:
    living_condition_df = pd.DataFrame(df)
    living_condition_df.rename(columns={'value' : living_condition_api[i]}, inplace=True)

In [ ]:

urls = []
for indicator in agriculture_practice_api:
    url = 'http://api.worldbank.org/v2/countries/indicators/' + indicator 
    urls.append(url)
dataframe_list = []

for url in urls:
    data = []
    try:  
        for page in range(1,18):
            payload = {'format': 'json', 'per_page': '1000', 'date':'1960:2022', 'page':page}     
            r = requests.get(url, params=payload)
            data+=r.json()[1]
        
        dataframe_list.append(pd.DataFrame(data))

    except:
        print('could not load data', url)

In [ ]:
agricultural_practice_df = None

#format and combine datframes into one
for i, df in enumerate(dataframe_list):
  dataframe_list[i] = removeFormatColumns(df)
  dataframe_list[i] = formatColumns(df)
  dataframe_list[i] = justCountries(df)
  
  if agricultural_practice_df is not None:
    agricultural_practice_df.insert(loc=len(agricultural_practice_df.columns),column=agriculture_practice_api[i], 
    value=df['value'])
  else:
    agricultural_practice_df = pd.DataFrame(df)
    agricultural_practice_df.rename(columns={'value' : agriculture_practice_api[i]}, inplace=True)

In [ ]:
#countries with the top 10 populations
largest_10_population = countries_population_df[countries_population_df['date']=='2021'].nlargest(n=10,columns='value')

largest_10_list = [country for country in largest_10_population['country']]

largest_10_pop_df = countries_population_df[countries_population_df['country'].isin(largest_10_list)]